In [3]:
#importing libraries
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
from langchain.document_loaders import PyPDFLoader,DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers
import os
import uuid
import pinecone


c:\End to End Projects\Medical Chatbot Gen AI\menv\lib\site-packages\pinecone\index.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [4]:
#extract data from pdf

loader = DirectoryLoader(
    path='C:\End to End Projects\Medical Chatbot Gen AI\data',
    glob='*.pdf',
    loader_cls=PyPDFLoader
    )
documents = loader.load()

In [6]:
#creating text chunks

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=20)

text_chunks = text_splitter.split_documents(documents)

In [7]:
len(text_chunks)

7020

In [8]:
#embedding model

embedding = HuggingFaceEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2')

In [9]:
embedding

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={})

In [10]:
query = embedding.embed_query('Hello')
print(len(query))

384


In [11]:
PINECONE_API_KEY = "aca07530-5144-453c-ac16-318eebed1265"
PINECONE_API_ENV = "gcp-starter"

In [12]:
#initializing pinecone
pinecone.init(api_key=PINECONE_API_KEY,
              environment=PINECONE_API_ENV)


In [14]:
index_name="medical-chatbot"

In [15]:
#Creating Embeddings for Each of The Text Chunks & storing
Docsearch = Pinecone.from_texts([t.page_content for t in text_chunks],
                                embedding,
                                index_name=index_name)